In [ ]:
import pandas as pd
import dataframeLoader as dfl
import plotly

root = '.dataDir'
## load timeseries data from CSV files
df10 = dfl.loadStrucDataFromFileRegex(root, 'STRUCTURED-*.csv')
df = dfl.loadPrometheusDataFromFileRegex(root, 'securiti_appliance_', '.csv')
df9 = dfl.loadUnstrucDataFromFileRegex(root, 'UNSTRUCTURED-*.csv')

df = pd.concat([df, df9, df10], ignore_index=True)

display(df)

In [ ]:
## Generate top performing appliances between time-period.
fromDt = '2024-08-05'
toDt = '2024-09-09'
df10 = dfl.loadDataFrameFromFileRegex(root, 'UNSTRUCTURED-*.csv', metrics='dataScanned')
df10['ts']=pd.to_datetime(df10['ts'],unit='ms')
df10 = df10[(df10['ts'] >= fromDt) & (df10['ts'] <= toDt) ]
df10 = df10.groupby(['tenant', 'pod']).agg(dataScanned=('dataScannedInGB', 'sum')).reset_index()
df10 = df10.sort_values('dataScanned', ascending=False)
display(df10.head(20))


In [15]:
# from pathlib import Path
# Path("appliancePlots").mkdir(parents=True, exist_ok=True)
# for tenant, appliance in df10.head(20).iterrows():
#     ttl = appliance.tenant+'-'+appliance.pod
#     ## Generate timeseries fascet graph. 
#     plotly.offline.plot(dfl.plotMetricsFacetForApplianceId(df, appliance.pod, fromDt, toDt, ttl), filename='appliancePlots/'+ttl+'.html')

In [ ]:
## Generate timeseries fascet graph for an appliance_id
fig  = dfl.plotMetricsFacetForApplianceId(df, '58e98e10-1b19-4c84-93c0-db2ad5903b80', '2024-08-05', '2024-09-09', "Cluster 58e98e10-1b19-4c84-93c0-db2ad5903b80")
# fig.update_yaxes(type="log")
fig.show()